In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim


In [2]:
quantile = 0.9
n_samples = 10000
n_features = 30
n_miss_features = 5
interval_length = 100
np.random.seed(0)

train_ratio = 0.6
validation_ratio = 0.2
test_ratio = 0.2


# np.random.seed(0)  
# n_samples = 10000
# n_features = 30
# X = np.random.normal(0, 100, (n_samples, n_features))
# coefficients = np.random.normal(10, 400, n_features)
# noise = np.random.normal(0, 16, n_samples)
# Y = np.dot(X, coefficients) + noise


# 1. linear demand
$D = b_0 + \beta^{T} * X + \epsilon, $
where $X$ is the observed feature, and $\epsilon$ is a random noise centered at 0. 

In [3]:
X = np.random.normal(0, 100, (n_samples, n_features))
coefficients = np.random.normal(10, 400, n_features)
noise = np.random.normal(0, 16, n_samples)
## noise with bias

Y = np.dot(X, coefficients) + noise

X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=1 - train_ratio, random_state=0)
X_test, X_validation, Y_test, Y_validation = train_test_split(X_temp, Y_temp, test_size=validation_ratio/(test_ratio + validation_ratio), random_state=0)

In [4]:
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

quantile_reg_model = sm.QuantReg(Y_train, X_train_const).fit(q=quantile)
linear_model = sm.OLS(Y_train, X_train_const).fit()

Y_pred_test = quantile_reg_model.predict(X_test_const)
Y_linear_pred_test = linear_model.predict(X_test_const)

E_i = Y_test - Y_pred_test
E_i_linear = Y_test - Y_linear_pred_test

adjusted_quantile = quantile * (1 + 1 / len(E_i))
adjusted_quantile_linear = quantile*(1 + 1 / len(E_i_linear))

Q_alpha_E = np.quantile(E_i, adjusted_quantile)
Q_alpha_E_linear = np.quantile(E_i_linear, adjusted_quantile_linear)

In [5]:
def calculate_quantile(Y_train, Y_validation, alpha):
    Y_pred_online_SAA= []
    for i in range(len(Y_validation)):
        combined_data = np.concatenate([Y_train, Y_validation[:i]])
        quantile = np.quantile(combined_data, alpha)
        Y_pred_online_SAA.append(quantile)
    return Y_pred_online_SAA


Y_pred_test_SAA = calculate_quantile(Y_train, Y_test, quantile)
E_SAA = Y_test - Y_pred_test_SAA
adjusted_quantile_SAA = quantile * (1 + 1 / len(E_SAA))
Q_SAA = np.quantile(E_SAA, adjusted_quantile_SAA)

In [6]:
X_validation_const = sm.add_constant(X_validation)

Y_pred_validation = quantile_reg_model.predict(X_validation_const)
Y_pred_validation_adjusted = Y_pred_validation + Q_alpha_E
Y_pred_validation_linear = linear_model.predict(X_validation_const)
Y_pred_validation_linear_adjusted = Y_pred_validation_linear + Q_alpha_E_linear

Y_pred_SAA = np.quantile(Y_train, quantile)
Y_pred_validation_online_SAA = calculate_quantile(Y_train, Y_validation, quantile)
Y_SAA_adjusted = Y_pred_validation_online_SAA + Q_SAA

In [7]:
def quantile_loss(y_true, y_pred, quantile):
    error = y_true - y_pred
    return np.maximum(quantile * error, (quantile - 1) * error).mean()


quantile_loss_unadjusted = quantile_loss(Y_validation, Y_pred_validation, quantile)
quantile_loss_adjusted = quantile_loss(Y_validation, Y_pred_validation_adjusted, quantile)
quantile_loss_linear = quantile_loss(Y_validation, Y_pred_validation_linear, quantile)
quantile_loss_linear_adjusted = quantile_loss(Y_validation, Y_pred_validation_linear_adjusted, quantile)
quantile_loss_SAA = quantile_loss(Y_validation, Y_pred_SAA, quantile)
quantile_loss_online_SAA = quantile_loss(Y_validation, Y_pred_validation_online_SAA, quantile)
quantile_loss_SAA_adjusted = quantile_loss(Y_validation, Y_SAA_adjusted, quantile)

print("Quantile Loss Unadjusted:", quantile_loss_unadjusted, "\n",  "Quantile Loss Adjusted:", quantile_loss_adjusted, "\n", 
      "Linear loss: ", quantile_loss_linear,  "\n","Linear adjusted loss:", quantile_loss_linear_adjusted, "\n", 
      "SAA quantile loss: ", quantile_loss_SAA, "\n", "online SAA quantile loss:", quantile_loss_online_SAA, "\n", 
      "SAA adjusted:", quantile_loss_SAA_adjusted)

Quantile Loss Unadjusted: 2.86939285813291 
 Quantile Loss Adjusted: 2.8683845624998523 
 Linear loss:  6.066188004836474 
 Linear adjusted loss: 2.860102686304872 
 SAA quantile loss:  37874.081829949646 
 online SAA quantile loss: 37878.38534957783 
 SAA adjusted: 37880.412719294916


# 2. linear demand with misspicification

$D = \beta * (X, X_1) + \epsilon, $
where $X$ could be observed, and $X_1$ couldn't

In [8]:
X = np.random.normal(0, 100, (n_samples, n_features))
X1 = np.random.normal(1, 16, (n_samples, n_miss_features))
X = np.concatenate((X, X1), axis=1)
coefficients = np.random.normal(10, 400, n_features + n_miss_features)
lower_bound = -interval_length * quantile
higher_bound = interval_length + lower_bound
noise = np.random.normal(lower_bound, higher_bound, n_samples)

Y = np.dot(X, coefficients) + noise

X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=1 - train_ratio, random_state=0)
X_test, X_validation, Y_test, Y_validation = train_test_split(X_temp, Y_temp, test_size=validation_ratio/(test_ratio + validation_ratio), random_state=0)

X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

quantile_reg_model = sm.QuantReg(Y_train, X_train_const).fit(q=quantile)
linear_model = sm.OLS(Y_train, X_train_const).fit()

Y_pred_test = quantile_reg_model.predict(X_test_const)
Y_linear_pred_test = linear_model.predict(X_test_const)

E_i = Y_test - Y_pred_test
E_i_linear = Y_test - Y_linear_pred_test

adjusted_quantile = quantile * (1 + 1 / len(E_i))
adjusted_quantile_linear = quantile*(1 + 1 / len(E_i_linear))

Q_alpha_E = np.quantile(E_i, adjusted_quantile)
Q_alpha_E_linear = np.quantile(E_i_linear, adjusted_quantile_linear)

Y_pred_test_SAA = calculate_quantile(Y_train, Y_test, quantile)
E_SAA = Y_test - Y_pred_test_SAA
adjusted_quantile_SAA = quantile * (1 + 1 / len(E_SAA))
Q_SAA = np.quantile(E_SAA, adjusted_quantile_SAA)

X_validation_const = sm.add_constant(X_validation)

Y_pred_validation = quantile_reg_model.predict(X_validation_const)
Y_pred_validation_adjusted = Y_pred_validation + Q_alpha_E
Y_pred_validation_linear = linear_model.predict(X_validation_const)
Y_pred_validation_linear_adjusted = Y_pred_validation_linear + Q_alpha_E_linear

Y_pred_SAA = np.quantile(Y_train, quantile)
Y_pred_validation_online_SAA = calculate_quantile(Y_train, Y_validation, quantile)
Y_SAA_adjusted = Y_pred_validation_online_SAA + Q_SAA

quantile_loss_unadjusted = quantile_loss(Y_validation, Y_pred_validation, quantile)
quantile_loss_adjusted = quantile_loss(Y_validation, Y_pred_validation_adjusted, quantile)
quantile_loss_linear = quantile_loss(Y_validation, Y_pred_validation_linear, quantile)
quantile_loss_linear_adjusted = quantile_loss(Y_validation, Y_pred_validation_linear_adjusted, quantile)
quantile_loss_SAA = quantile_loss(Y_validation, Y_pred_SAA, quantile)
quantile_loss_online_SAA = quantile_loss(Y_validation, Y_pred_validation_online_SAA, quantile)
quantile_loss_SAA_adjusted = quantile_loss(Y_validation, Y_SAA_adjusted, quantile)

print("Quantile Loss Unadjusted:", quantile_loss_unadjusted, "\n",  "Quantile Loss Adjusted:", quantile_loss_adjusted, "\n", 
      "Linear loss: ", quantile_loss_linear,  "\n","Linear adjusted loss:", quantile_loss_linear_adjusted, "\n", 
      "SAA quantile loss: ", quantile_loss_SAA, "\n", "online SAA quantile loss:", quantile_loss_online_SAA, "\n", 
      "SAA adjusted:", quantile_loss_SAA_adjusted)

Quantile Loss Unadjusted: 1.746321879888141 
 Quantile Loss Adjusted: 1.7493781914612159 
 Linear loss:  4.081985536538825 
 Linear adjusted loss: 1.7498799482888068 
 SAA quantile loss:  46065.963680330395 
 online SAA quantile loss: 46058.454414469496 
 SAA adjusted: 46014.00206041387


# 3. jackknife+ and conformal newsvendor

# 4. comparison with KO and NERV-1 and NERV-2

# 5. GLM demand


In [ ]:
X = np.random.normal(0, 100, (n_samples, n_features))
coefficients = np.random.normal(10, 400, n_features)
noise = np.random.normal(0, 16, n_samples)
Y = (np.dot(X, coefficients) + noise)

X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=1 - train_ratio, random_state=0)
X_test, X_validation, Y_test, Y_validation = train_test_split(X_temp, Y_temp, test_size=validation_ratio/(test_ratio + validation_ratio), random_state=0)

X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

quantile_reg_model = sm.QuantReg(Y_train, X_train_const).fit(q=quantile)
linear_model = sm.OLS(Y_train, X_train_const).fit()

Y_pred_test = quantile_reg_model.predict(X_test_const)
Y_linear_pred_test = linear_model.predict(X_test_const)

E_i = Y_test - Y_pred_test
E_i_linear = Y_test - Y_linear_pred_test

adjusted_quantile = quantile * (1 + 1 / len(E_i))
adjusted_quantile_linear = quantile*(1 + 1 / len(E_i_linear))

Q_alpha_E = np.quantile(E_i, adjusted_quantile)
Q_alpha_E_linear = np.quantile(E_i_linear, adjusted_quantile_linear)

Y_pred_test_SAA = calculate_quantile(Y_train, Y_test, quantile)
E_SAA = Y_test - Y_pred_test_SAA
adjusted_quantile_SAA = quantile * (1 + 1 / len(E_SAA))
Q_SAA = np.quantile(E_SAA, adjusted_quantile_SAA)

X_validation_const = sm.add_constant(X_validation)

Y_pred_validation = quantile_reg_model.predict(X_validation_const)
Y_pred_validation_adjusted = Y_pred_validation + Q_alpha_E
Y_pred_validation_linear = linear_model.predict(X_validation_const)
Y_pred_validation_linear_adjusted = Y_pred_validation_linear + Q_alpha_E_linear

Y_pred_SAA = np.quantile(Y_train, quantile)
Y_pred_validation_online_SAA = calculate_quantile(Y_train, Y_validation, quantile)
Y_SAA_adjusted = Y_pred_validation_online_SAA + Q_SAA

quantile_loss_unadjusted = quantile_loss(Y_validation, Y_pred_validation, quantile)
quantile_loss_adjusted = quantile_loss(Y_validation, Y_pred_validation_adjusted, quantile)
quantile_loss_linear = quantile_loss(Y_validation, Y_pred_validation_linear, quantile)
quantile_loss_linear_adjusted = quantile_loss(Y_validation, Y_pred_validation_linear_adjusted, quantile)
quantile_loss_SAA = quantile_loss(Y_validation, Y_pred_SAA, quantile)
quantile_loss_online_SAA = quantile_loss(Y_validation, Y_pred_validation_online_SAA, quantile)
quantile_loss_SAA_adjusted = quantile_loss(Y_validation, Y_SAA_adjusted, quantile)

print("Quantile Loss Unadjusted:", quantile_loss_unadjusted, "\n",  "Quantile Loss Adjusted:", quantile_loss_adjusted, "\n", 
      "Linear loss: ", quantile_loss_linear,  "\n","Linear adjusted loss:", quantile_loss_linear_adjusted, "\n", 
      "SAA quantile loss: ", quantile_loss_SAA, "\n", "online SAA quantile loss:", quantile_loss_online_SAA, "\n", 
      "SAA adjusted:", quantile_loss_SAA_adjusted)